In [1]:
# Operações de Crédito - Conta 16000001 (filtragem maiores saldos)

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancetes
    WHERE
        "CONTA" = '16000001'
)
SELECT
    *
FROM
    RankedData
WHERE
    SALDO > 1000000
--    rn <= 10;
"""

model = pd.read_sql_query(query, conn)

conn.close()

model = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'SALDO']]
model['cnpj'] = model['cnpj'].astype(int)
model = model.rename(columns={'SALDO': 'operacoes_de_credito'})


In [2]:
# Receita total
'''
71000008	RECEITAS OPERACIONAIS
73000006	RECEITAS NAO OPERACIONAIS
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71000008', '73000006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)

conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

In [3]:
# Ativo total (ln_TA)
'''
39999993	TOTAL GERAL DO ATIVO
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('39999993')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)

conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

model['ln_ta'] = np.log(model['ativo_total'])


In [4]:
# p - preco

model['p'] = model['receita_total']/model['ativo_total']


In [5]:
# Custo Total (ln_c)
'''
81100008	(-) Despesas De Captacao
81500000	(-) Despesas Com Titulos E Valores Mobiliarios E Instrumentos Financeiros Derivativos
81700006	(-) Despesas Administrativas
81800009	(-) Aprovisionamentos E Ajustes Patrimoniais
83000003	(-) DESPESAS NAO OPERACIONAIS
89400009	(-) Imposto De Renda
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81500000', '81700006', '81800009', '83000003', '89400009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'custo_total'})
table['custo_total'] = -table['custo_total']

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

model['ln_c'] = np.log(model['custo_total'])


/home/hspassos/anaconda3/envs/bcb/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
# w_1 - Custo Administrativo
'''
81700006	(-) Despesas Administrativas
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81700006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'custo_administrativo'})
table['custo_administrativo'] = -table['custo_administrativo']

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


model['w1'] = 100* model['custo_administrativo'] / model['ativo_total']


In [7]:
# Despesa Operacional
'''
81000005	(-) DESPESAS OPERACIONAIS
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81000005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_operacional'})
table['despesa_operacional'] = -table['despesa_operacional']

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [8]:
# Ativo Fixo
'''
21000003	INVESTIMENTOS
22000002	IMOBILIZADO DE USO
25000009	INTANGIVEL
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('21000003', '22000002', '25000009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_fixo'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [9]:
# w_2 - Preço do Capital Fixo

model['w2'] = (model['despesa_operacional'] - model['custo_administrativo']) / model['ativo_fixo']

In [10]:
# Despesa Financeira
'''
81100008	(-) Despesas De Captacao
81200001	(-) Despesas De Obrigacoes Por Emprestimos E Repasses
81500000	(-) Despesas Com Titulos E Valores Mobiliarios E Instrumentos Financeiros Derivativos
81800009	(-) Aprovisionamentos E Ajustes Patrimoniais
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001', '81500000', '81800009')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'custo_financeiro'})
table['custo_financeiro'] = -table['custo_financeiro']

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [11]:
# Depósitos
'''
41000007	DEPOSITOS
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('41000007')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'deposito_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [12]:
# w_3 - Preço dos depósitos

model['w3'] = model['custo_financeiro'] / model['deposito_total']


In [13]:
# índice de Lerner

lerner = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'ln_c', 'ln_ta', 'w1', 'w2', 'w3']]
lerner.loc[:, 'ln_ta**2'] = (lerner['ln_ta']**2)*(1/2)
lerner.dropna(subset=['ln_c', 'ln_ta', 'ln_ta**2', 'w1', 'w2', 'w3'], inplace=True)

if np.isinf(lerner[['ln_c', 'ln_ta', 'ln_ta**2', 'w1', 'w2', 'w3']]).any().any():
    print("Aviso: Valores infinitos encontrados e removidos.")
    lerner = lerner.replace([np.inf, -np.inf], np.nan).dropna(subset=['ln_c', 'ln_ta', 'ln_ta**2', 'w1', 'w2', 'w3'], how='any')

import statsmodels.api as sm

lerner_x = lerner[['ln_ta', 'ln_ta**2', 'w1', 'w2', 'w3']]
lerner_y = lerner['ln_c']

reg = sm.OLS(lerner_y, sm.add_constant(lerner_x)).fit()
results_lerner = reg
print(results_lerner.summary())

'''
spread_estimado = lerner[['data', 'cnpj', 'NOME_INSTITUICAO']]
spread_estimado['spread_estimado'] = results_stage1.predict()
spread_estimado['spread_puro'] = results_stage1.params['const'] + lerner[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])
'''

/tmp/ipykernel_26540/3217596091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lerner.loc[:, 'ln_ta**2'] = (lerner['ln_ta']**2)*(1/2)
/tmp/ipykernel_26540/3217596091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lerner.dropna(subset=['ln_c', 'ln_ta', 'ln_ta**2', 'w1', 'w2', 'w3'], inplace=True)


Aviso: Valores infinitos encontrados e removidos.
                            OLS Regression Results                            
Dep. Variable:                   ln_c   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                 2.116e+05
Date:                Thu, 13 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:42:37   Log-Likelihood:            -1.6413e+05
No. Observations:              134856   AIC:                         3.283e+05
Df Residuals:                  134850   BIC:                         3.283e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co

"\nspread_estimado = lerner[['data', 'cnpj', 'NOME_INSTITUICAO']]\nspread_estimado['spread_estimado'] = results_stage1.predict()\nspread_estimado['spread_puro'] = results_stage1.params['const'] + lerner[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])\n"

In [14]:
# HHI

model['credito_total'] = model['operacoes_de_credito'].groupby(model['data']).transform('sum')
model['mktsh'] = 100 * (model['operacoes_de_credito'] / model['credito_total'])

In [15]:
# AOC (Average Operating Costs)

model['aoc'] = model['despesa_operacional'] / model['ativo_total']



In [16]:
# STD - Desvio padrão taxas a termo

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    std_taxas_termo
"""

taxas_termo = pd.read_sql_query(query, conn)

conn.close()

model = pd.merge(
    model,
    taxas_termo,
    on=['data'],
    how='left'
)

In [17]:
# Provisões de crédito (calcular diff?)
'''
16900008	(-) Provisoes Para Operacoes De Credito

81800009	(-) Aprovisionamentos E Ajustes Patrimoniais (?)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16900008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table['saldo_total_agregado'] = -table['saldo_total_agregado']
table = table.rename(columns={'saldo_total_agregado': 'provisao_credito'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [18]:
# CRERISK - Risco de Crédito

model['crerisk'] = model['provisao_credito'] / model['operacoes_de_credito']


In [19]:
# STD*CRERISK - Interação dos Riscos

model['std_3m*crerisk'] = model['std_3m'] * model['crerisk']
model['std_1a*crerisk'] = model['std_1a'] * model['crerisk']


In [20]:
#Patrimônio Liquido
'''
60000002	PATRIMONIO LIQUIDO
'''
conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('60000002')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'patrimonio_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)




In [21]:
# RISKAVER

model['riskaver'] = model['patrimonio_liquido'] / model['ativo_total']


In [22]:
# SIZE: Volume de empréstimos concedidos (em logaritmo)

#model['size'] = model['operacoes_de_credito'].diff
model['size'] = np.log10(model['operacoes_de_credito'])


In [23]:
# Receita não financeira
'''
71700009	Rendas De Prestacao De Servicos
71900005	Outras Receitas Operacionais
73000006	RECEITAS NAO OPERACIONAIS
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
        "CONTA" IN ('71700009', '73000006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_nao_financeira'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [24]:
# IIP - Implicit Interest Payments

model['iip'] = (model['despesa_operacional'] - model['receita_nao_financeira']) / model['ativo_total']


In [25]:
# Ativos liquidos (Caixa e Equivalentes de Caixa)
'''
Caixa (1.1.1.00.00-9)
Depósitos Bancários (1.1.2.00.00-2)
Reservas Livres (1.1.3.00.00-5)
Aplicações em Operações Compromissadas (1.2.1.00.00-8)
Aplicações em Depósitos Interfinanceiros (1.2.2.00.00-1)
Disponibilidades em Moedas Estrangeiras (1.1.5.00.00-1)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11200002', '11300005', '12100008', '12200001', '11500001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [26]:
# RESER - Custo de Oportunidade das Reservas

model['reser'] = model['ativo_liquido'] / model['ativo_total']


In [27]:
# Receita Operacional
'''
71000008	RECEITAS OPERACIONAIS
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71000008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_operacional'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)




In [28]:
# EF - Efficiency

model['ef'] = model['despesa_operacional'] / model['receita_operacional']


In [29]:
# Receitas de Juros e Similares (Rendas)
'''
Rendas De Operacoes De Credito (7.1.1.00.00-1)
Rendas de Arrendamento Mercantil (7.1.2.00.00-4)
Rendas de Aplicações Interfinanceiras de Liquidez (7.1.4.00.00-0)
Rendas de Títulos e Valores Mobiliários (7.1.5.00.00-3)
Rendas de Outras Operações Com Características de Crédito (7.1.6.00.00-8)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71100001', '71200004', '71400000', '71500003', '71600008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_juros'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [30]:
# Despesas de Juros e Encargos (Custos de Captação)
'''
Despesas de captação (81100008)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_captacao'})
table['despesa_captacao'] = table['despesa_captacao'] * (-1)

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

model['log_despesa_captacao'] = np.log10(model['despesa_captacao'])

In [31]:
# Ativos Rentáveis
'''
Operações de Crédito (1.6.0.00.00-1)
Títulos e Valores Mobiliários (1.3.0.00.00-4)
Arrendamento Mercantil (1.7.0.00.00-1)
12000005	APLICACOES INTERFINANCEIRAS DE LIQUIDEZ
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16000001', '13000004', '17000001', '12000005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativos_rentaveis'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [32]:
# NIM

model['nim'] = 100 * (model['receita_juros'] - model['despesa_captacao']) / model['ativos_rentaveis']

In [39]:
# Criando tabela variables

variables = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'nim', 'mktsh', 'aoc', 'riskaver','std_3m', 'std_1a', 'std_2a', 'std_3a', 'std_5a', 'crerisk', 'std_3m*crerisk', 'std_1a*crerisk', 'size', 'iip', 'reser', 'ef']]
variables.insert(loc=0, column='date', value= pd.to_datetime(variables['data']))

# Variáveis lags
'''
coluna_grupo = 'cnpj'
colunas_para_verificar = ['nibra', 'nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr', 'mktsh']

for col in colunas_para_verificar:
    variables[f'{col}_lag_2'] = (variables.groupby(coluna_grupo)[col].shift(2))
    variables[f'{col}_lag_1'] = (variables.groupby(coluna_grupo)[col].shift(1))
    variables[f'{col}_lag_3'] = (variables.groupby(coluna_grupo)[col].shift(3))
'''
# Dummies de tempo
dummies_tempo = pd.get_dummies(variables['data'], drop_first=True)
dummies_tempo = dummies_tempo.astype(int)
variables = pd.concat([variables, dummies_tempo], axis=1)

# Removendo linhas com valores nulos, exceto na coluna NOME_INSTITUICAO
todas_colunas = variables.columns.tolist()
colunas_para_verificar = [col for col in todas_colunas if col != 'NOME_INSTITUICAO']
variables = variables.dropna(how='any', subset=colunas_para_verificar)


In [ ]:
# Tratando outliers usando o método do IQR (Interquartile Range)
colunas_para_analisar = ['nim', 'aoc', 'riskaver', 'crerisk', 'iip', 'reser', 'ef']

# True significa que a linha *será removida* (outlier).
# Começamos com False, assumindo que nenhuma linha será removida inicialmente.
mascara_outliers_acumulada = pd.Series(False, index=variables.index)

print(f"Tamanho do DataFrame antes da remoção: {len(variables)}")

for col in colunas_para_analisar:
    Q1 = variables[col].quantile(0.10)
    Q3 = variables[col].quantile(0.9e0)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    mascara_outliers_coluna = (variables[col] < limite_inferior) | (variables[col] > limite_superior)

    mascara_outliers_acumulada = mascara_outliers_acumulada | mascara_outliers_coluna

    num_outliers_col = mascara_outliers_coluna.sum()
    print(f"Coluna '{col}': {num_outliers_col} outliers identificados (Limites: [{limite_inferior:.2f}, {limite_superior:.2f}])")


# Criamos uma máscara de linhas *a serem mantidas* (True = não é outlier)
mascara_a_manter = ~mascara_outliers_acumulada # O til (~) inverte a máscara
variables_limpo = variables[mascara_a_manter]

variables_limpo.loc[:, 'aoc'] = 100*variables_limpo['aoc']
variables_limpo.loc[:, 'riskaver'] = 100*variables_limpo['riskaver']
variables_limpo.loc[:, 'std_3m'] = 100*variables_limpo['std_3m']
variables_limpo.loc[:, 'std_1a'] = 100*variables_limpo['std_1a']
variables_limpo.loc[:, 'std_2a'] = 100*variables_limpo['std_2a']
variables_limpo.loc[:, 'std_3a'] = 100*variables_limpo['std_3a']
variables_limpo.loc[:, 'std_5a'] = 100*variables_limpo['std_5a']
variables_limpo.loc[:, 'crerisk'] = 100*variables_limpo['crerisk']
variables_limpo.loc[:, 'std_3m*crerisk'] = variables_limpo['std_3m']*variables_limpo['crerisk']
variables_limpo.loc[:, 'std_1a*crerisk'] = variables_limpo['std_1a']*variables_limpo['crerisk']
variables_limpo.loc[:, 'iip'] = 100*variables_limpo['iip']
variables_limpo.loc[:, 'reser'] = 100*variables_limpo['reser']
variables_limpo.loc[:, 'ef'] = 100*variables_limpo['ef']

print(f"\nTotal de linhas removidas: {mascara_outliers_acumulada.sum()}")
print(f"Tamanho do DataFrame após a remoção: {len(variables_limpo)}")


Tamanho do DataFrame antes da remoção: 135607
Coluna 'nim': 3753 outliers identificados (Limites: [-13.58, 24.69])
Coluna 'aoc': 1270 outliers identificados (Limites: [-0.06, 0.11])
Coluna 'riskaver': 2490 outliers identificados (Limites: [-0.20, 0.40])
Coluna 'crerisk': 4425 outliers identificados (Limites: [-0.16, 0.31])
Coluna 'iip': 1428 outliers identificados (Limites: [-0.05, 0.10])
Coluna 'reser': 4759 outliers identificados (Limites: [-0.06, 0.11])
Coluna 'ef': 1752 outliers identificados (Limites: [0.19, 1.59])

Total de linhas removidas: 15254
Tamanho do DataFrame após a remoção: 120353


In [42]:
# Regressão
import statsmodels.api as sm

colunas_X_originais = ['mktsh', 'aoc', 'riskaver','std_3m', 'std_1a', 'std_3a', 'std_5a', 'crerisk', 'size', 'iip', 'reser', 'ef']
colunas_X = colunas_X_originais + list(dummies_tempo.columns)
X = variables_limpo[colunas_X]
y = variables_limpo['nim']

stage1 = sm.OLS(y, sm.add_constant(X)).fit()
results_stage1 = stage1
print(results_stage1.summary())

spread_estimado = variables_limpo[['data', 'cnpj', 'NOME_INSTITUICAO']]
spread_estimado['spread_estimado'] = results_stage1.predict()
spread_estimado['spread_puro'] = results_stage1.params['const'] + variables_limpo[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])


                            OLS Regression Results                            
Dep. Variable:                    nim   R-squared:                       0.422
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     469.4
Date:                Thu, 13 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:46:33   Log-Likelihood:            -2.9767e+05
No. Observations:              120353   AIC:                         5.957e+05
Df Residuals:                  120165   BIC:                         5.975e+05
Df Model:                         187                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.8789      0.258      7.271      0.0

/tmp/ipykernel_26540/1258644758.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread_estimado['spread_estimado'] = results_stage1.predict()
/tmp/ipykernel_26540/1258644758.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread_estimado['spread_puro'] = results_stage1.params['const'] + variables_limpo[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])


In [36]:
# Regressões por ano

variables_limpo['ANO'] = variables_limpo['date'].dt.year
resultados_por_ano = {}

print("--- Iniciando Regressão OLS por Ano ---")
for ano, dados_anuais in variables_limpo.groupby('ANO'):
    print(f"\n--- Ajustando Regressão para o Ano: {ano} ---")

    X_anual = dados_anuais[colunas_X]
    y_anual = dados_anuais['nim']

    df_temp = pd.concat([y_anual, X_anual], axis=1).dropna()
    X_final = df_temp.drop(columns=['nim'])
    y_final = df_temp['nim']

    # Garantir que há dados suficientes para a regressão
    if len(y_final) > len(X_final.columns) + 1: # Pelo menos k+1 observações
        
        # C. Adicionar a constante e ajustar o modelo
        X_com_const = sm.add_constant(X_final)
        
        try:
            regressao = sm.OLS(y_final, X_com_const).fit()

            resultados_por_ano[ano] = regressao
          
            # Imprimir um resumo simples (ou o summary completo, se desejar)
            print(f"R-quadrado: {regressao.rsquared:.4f}")
            print(f"Coeficientes Significativos:\n{regressao.pvalues[regressao.pvalues < 0.05]}")
            
        except Exception as e:
            print(f"Erro ao ajustar o modelo para {ano}: {e}")
    else:
        print(f"Dados insuficientes para ajustar o modelo no ano {ano}.")

# 5. Exibir os resultados de um ano específico (exemplo)
if 2017 in resultados_por_ano: # Altere o ano conforme sua base
    print("\n\n--- Resumo Detalhado da Regressão (Ano 2017) ---")
    print(resultados_por_ano[2017].summary())


--- Iniciando Regressão OLS por Ano ---


/tmp/ipykernel_26540/2344732754.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables_limpo['ANO'] = variables_limpo['date'].dt.year



--- Ajustando Regressão para o Ano: 2010 ---
R-quadrado: 0.3601
Coeficientes Significativos:
const       9.158818e-07
aoc         2.751411e-13
riskaver    3.284330e-14
std_3m      7.986654e-43
std_1a      1.737733e-23
crerisk     1.383328e-70
size        4.565899e-04
iip         1.036680e-03
reser       5.217622e-03
ef          5.708902e-17
2010-03     2.338486e-04
2010-04     2.727925e-04
2010-05     1.848211e-16
2010-06     1.024853e-33
2010-07     9.041028e-19
2010-09     9.660502e-03
2010-10     3.860964e-11
2010-11     1.209864e-07
2010-12     3.375667e-24
dtype: float64

--- Ajustando Regressão para o Ano: 2011 ---
R-quadrado: 0.3757
Coeficientes Significativos:
const       6.053360e-14
aoc         1.776787e-20
riskaver    6.377716e-19
std_1a      7.898602e-08
crerisk     3.264862e-60
iip         1.047019e-06
reser       1.677489e-08
ef          2.600524e-15
2010-02     2.629917e-11
2010-03     1.386959e-12
2010-04     3.938076e-12
2010-05     2.763448e-16
2010-06     5.934471e-

In [37]:
variables_limpo_2010 = variables_limpo[variables_limpo['date'] == 2010]